In [1]:
import pandas as pd
import ast
import os
import redis
os.chdir('..')

# Get the data into pandas

In [2]:
with open("data/log_tiguan_27_mar_dac.txt") as f:
    data = ast.literal_eval(f.read())

df = pd.DataFrame()
for data_value in data: 
    temp_df = pd.DataFrame(data_value[list(data_value)[0]]).sort_values(by="ts_millis:", ascending=True)["value"]
    temp_df.rename(list(data_value)[0], inplace=True)
    df = pd.concat([df, temp_df], axis = 1) 

In [7]:
df

,engine_load,engine_coolant_temp,engine_speed,vehicle_speed,intake_air_temp,maf,throttle_position,fuel_rg_pressure,barometric_pressure,control_voltage
0,37.25,17.0,904.5,0.0,10.0,12.55,83.14,37270.0,101.0,0.06
1,37.25,17.0,906.0,0.0,11.0,12.36,83.14,37800.0,101.0,14.56
2,36.86,17.0,905.0,0.0,10.0,12.36,83.53,37800.0,101.0,14.68
3,36.86,18.0,905.5,0.0,11.0,12.30,83.53,37800.0,101.0,14.72
4,36.86,18.0,907.0,0.0,11.0,12.47,83.14,36740.0,101.0,14.72
...,...,...,...,...,...,...,...,...,...,...
1030,42.35,90.0,2261.5,0.0,23.0,8.13,83.14,29740.0,101.0,11.92
1031,30.98,90.0,874.0,0.0,21.0,7.94,83.14,108770.0,101.0,11.82
1032,41.57,90.0,2282.5,20.0,24.0,30.86,83.14,30280.0,101.0,13.44
1033,30.98,90.0,873.0,0.0,25.0,7.83,83.14,31080.0,101.0,14.64


In [26]:
df.dropna(inplace=True)

In [30]:
df["class"] = 0
df.loc[:100, ["class"]] = 1
df.loc[:100, ["engine_load"]] = 100

In [36]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(df.drop(columns=["class"]), df["class"])

[LightGBM] [Info] Number of positive: 101, number of negative: 934
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 508
[LightGBM] [Info] Number of data points in the train set: 1035, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.097585 -> initscore=-2.224356
[LightGBM] [Info] Start training from score -2.224356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

LGBMClassifier()

/home/palo/work_dir/cttc/success6g-edge/success6g/lib/python3.8/site-packages/lightgbm/basic.py:696: UserWarning: Usage of np.ndarray subset (sliced data) is not recommended due to it will double the peak memory cost in LightGBM.
  _log_warning("Usage of np.ndarray subset (sliced data) is not recommended "


array([1])

# Redis

## Install and start the redis database

```
pip install prometheus-client
sudo apt install lsb-release curl gpg
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis

sudo service redis-server start
# sudo service redis-server stop
sudo service redis-server status
redis-cli ping
```

## Fill the Redis database
send data to redis
```
redis-cli -h localhost -p 6379
XADD v2x * car_id 123 engine_load 37.25 engine_coolant_temp 17.00 engine_speed 904.50 vehicle_speed 0.00 intake_air_temp 10.00 maf 12.55 throttle_position 83.14 fuel_rg_pressure 37270.00 barometric_pressure 101.00 control_voltage 0.0666
```

In [8]:
# Create connection object
redis_conn = redis.Redis(host='localhost', port=6379)
# Add some data into 'my_stream' as key-value pairs
# entry_data = {'field1': 'value1', 'field2': 'value2'}
# for index, row in df.iterrows():
#     redis_conn.xadd("v2x", dict(row))

## Retrieve data from redis

In [44]:
while True:
    # Listening for new items with XREAD
    # https://redis.io/docs/data-types/streams/
    [[stream, [[number, parts]]]] = redis_conn.xread(streams={"v2x":"$"}, block=0)
    keys = [p.decode('utf-8') for p in parts.keys()]
    vals = [float(p.decode('utf-8')) for p in parts.values()]
    d = pd.DataFrame(dict(zip(keys, vals)), index=[0])
    d["predicted_class"] = model.predict(d.drop(columns=["car_id"]))[0]
    print(d)

   car_id  engine_load  engine_coolant_temp  engine_speed  vehicle_speed  \
0   123.0        37.25                 17.0         904.5            0.0   

   intake_air_temp    maf  throttle_position  fuel_rg_pressure  \
0             10.0  12.55              83.14           37270.0   

   barometric_pressure  control_voltage  predicted_class  
0                101.0             0.09                0  
   car_id  engine_load  engine_coolant_temp  engine_speed  vehicle_speed  \
0   123.0         60.0                 17.0         904.5            0.0   

   intake_air_temp    maf  throttle_position  fuel_rg_pressure  \
0             10.0  12.55              83.14           37270.0   

   barometric_pressure  control_voltage  predicted_class  
0                101.0             0.09                0  
   car_id  engine_load  engine_coolant_temp  engine_speed  vehicle_speed  \
0   123.0         90.0                 17.0         904.5            0.0   

   intake_air_temp    maf  throttle_pos

KeyboardInterrupt: 

In [18]:
d

{'car_id': 123.0,
 'engine_load': 37.25,
 'engine_coolant_temp': 17.0,
 'engine_speed': 904.5,
 'vehicle_speed': 0.0,
 'intake_air_temp': 10.0,
 'maf': 12.55,
 'throttle_position': 83.14,
 'fuel_rg_pressure': 37270.0,
 'barometric_pressure': 101.0,
 'control_voltage': 0.0666}

In [14]:
d

{'engine_load': 37.25,
 'engine_coolant_temp': 17.0,
 'engine_speed': 904.5,
 'vehicle_speed': 0.0,
 'intake_air_temp': 10.0,
 'maf': 12.55,
 'throttle_position': 83.14,
 'fuel_rg_pressure': 37270.0,
 'barometric_pressure': 101.0,
 'control_voltage': 0.06}